# Imports & Dependencies

In [12]:
import pandas as pd
from collections import defaultdict
import sys
import os
import shutil as sh
import urllib
import tarfile
import numpy as np
import math
import seaborn as sns
import glob, os
import importlib
import gzip
import MDAnalysis as mda
import nglview as nv
import requests
import json
from biopandas.pdb import PandasPdb
from Bio import AlignIO
import re

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from scipy.spatial import ConvexHull

from urllib.error import HTTPError
from pathlib import Path
from ipywidgets import interact, interactive, fixed, interact_manual, IntProgress
import ipywidgets as widgets # type: ignore
from IPython.display import display, Markdown, clear_output

#Pandarallel works only on linux and mac
try:
    from pandarallel import pandarallel
    pandarallel.initialize(nb_workers=8,progress_bar=True)
    PARRALEL = True
except:
    PARRALEL = False

from tqdm.notebook import tnrange, tqdm
tqdm.pandas() #activate tqdm progressbar for pandas apply

#Pandas configuration
pd.options.mode.chained_assignment = (
    None  # default='warn', remove pandas warning when adding a new column
)

pd.set_option("display.max_columns", None)

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%config InlineBackend.figure_format ='svg' #better quality figure figure

#%matplotlib inline
sns.set_style("darkgrid")

np.seterr(divide='ignore', invalid='ignore')



INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


{'divide': 'ignore', 'over': 'warn', 'under': 'ignore', 'invalid': 'ignore'}

# Folder Creation

In [13]:
SETUP = {} #Dictionnary with ALL parameters

In [14]:
#Folder definition
from sys import platform
if platform == "linux" or platform == "linux2":
    PEPRMINT_FOLDER = "/home/user_stel/AISB/Project"
elif platform == "darwin":
    PEPRMINT_FOLDER = "/home/user_stel/AISB/Project"
else:
    raise ValueError("OS NOT FOUND")
WORKDIR = f"{PEPRMINT_FOLDER}/dataset/"
CATHFOLDER = f"{PEPRMINT_FOLDER}/databases/cath/"
ALPHAFOLDFOLDER = f"{PEPRMINT_FOLDER}/databases/alphafold/"
PROSITEFOLDER = f"{PEPRMINT_FOLDER}/databases/prosite/"
UNIPROTFOLDER = f"{PEPRMINT_FOLDER}/databases/uniprot/"
FIGURESFOLDER = f"{PEPRMINT_FOLDER}/figures/"

SETUP["PEPRMINT_FOLDER"]=PEPRMINT_FOLDER
SETUP["WORKDIR"]=WORKDIR
SETUP["CATHFOLDER"]=CATHFOLDER
SETUP["PROSITEFOLDER"]=PROSITEFOLDER
SETUP["ALPHAFOLDFOLDER"]=ALPHAFOLDFOLDER
SETUP["UNIPROTFOLDER"]=UNIPROTFOLDER
SETUP["FIGURESFOLDER"]=FIGURESFOLDER

In [15]:
if not os.path.exists(PEPRMINT_FOLDER):
    os.makedirs(PEPRMINT_FOLDER)
if not os.path.exists(WORKDIR):
    os.makedirs(WORKDIR)
if not os.path.exists(FIGURESFOLDER):
    os.makedirs(FIGURESFOLDER)
if not os.path.exists(ALPHAFOLDFOLDER):
    os.makedirs(ALPHAFOLDFOLDER)
if not os.path.exists(UNIPROTFOLDER):
    os.makedirs(UNIPROTFOLDER)
if not os.path.exists(PROSITEFOLDER):
    os.makedirs(PROSITEFOLDER) #MSA will contains the alignments in "msa" format (FASTA). 
if not os.path.exists(CATHFOLDER):
    os.makedirs(CATHFOLDER)

In [16]:
for k in SETUP:
    exec(f"{k}2 = SETUP['{k}']")

In [17]:
DOMAIN_PROSITE = {
    "PH": "PS50003",
    "C2": ["PS50004","PS51547"],
    "C1": "PS50081",  # Note : no C1 prosite on SMART but 2 C1 ProSite on Interprot (PS50081,PS00479), I took PS50081 since the data in PS00479 are in PS50081.
    "PX": "PS50195",
    # "FYVE":"PS50178",
    "FYVE": ["PS50178",'PS50089', 'PS00518','PS50016','PS01359','PS50014','PS00633','PS50119'],  # FYVE CAN BE THIS ONE TOO....
    # "PPASE_MYOTUBULARIN":"PS51339",# no GRAM domain found on prosite. Has to do this manually. Go on http://smart.embl-heidelberg.de/smart/do_annotation.pl?DOMAIN=GRAM&BLAST=DUMMY
    "BAR": "PS51021",  # 1URU is missing on prosite
    # "GLA":"PS50963",
    "ENTH": "PS50942",
    "SH2": "PS50001",
    "SEC14": "PS50191",
    "START": "PS50848",
    "C2DIS":"PS50022",
    "GLA": "PS50998",
    "PLD":"PS50035",
    "PLA":"PS00118",
    "ANNEXIN":"PS00223",
}
# Invert keys and values to have PROSITEID ==> DOMAIN
PROSITE_DOMAIN = {}
for key, value in DOMAIN_PROSITE.items():
    if type(value) == type([]):
        for subvalues in value:
            PROSITE_DOMAIN[subvalues] = key
    else:
        PROSITE_DOMAIN[value] = key
# PROSITE_DOMAIN = {v: k for k, v in DOMAIN_PROSITE.items()}

DOMAIN_CATH = {
    "PH": "2.30.29.30",
    "C2": "2.60.40.150",
    "C1": "3.30.60.20",
    "PX": "3.30.1520.10",
    "FYVE": "3.30.40.10",
    "BAR": "1.20.1270.60",
    "ENTH": "1.25.40.90",
    "SH2": "3.30.505.10",
    "SEC14": "3.40.525.10",
    "START": "3.30.530.20",
    "C2DIS": "2.60.120.260",
    "GLA":"2.40.20.10",
    "PLD":"3.20.20.190",
    "PLA":"1.20.90.10",
    "ANNEXIN":"1.10.220.10",
}

DOMAIN_INTERPRO = {
    "PH": "SSF50729",
    "C2": "SSF49562",
    "C1": None,
    "PX": "SSF64268",
    "FYVE": "SSF57903", #badly classified it looks like...
    "BAR": "SSF103657",
    "ENTH": "SSF48464",
    "SH2": "SSF55550",
    "SEC14": ["SSF52087","SSF46938"], #the CRAL TRIO domain is truncated in SSF.
    "START": "SSF55961",
    "C2DIS": "SSF49785",
    "GLA":None,
    "PLD":"SSF51695",
    "PLA":"G3DSA:1.20.90.10",
    "ANNEXIN":"SSF47874",
}

DOMAIN_INTERPRO_REFINE = {
    "PH": True,
    "C2": False,
    "C1": False,
    "PX": True,
    "FYVE": False,
    "BAR": False,
    "ENTH": False,
    "SH2": False,
    "SEC14": False,
    "START": True,
    "C2DIS": False,
    "GLA":False,
    "PLD":False,
    "PLA":True,
    "ANNEXIN":False,
}

# Invert keys and values to have CATHID ==> DOMAIN
CATH_DOMAIN = {v: k for k, v in DOMAIN_CATH.items()}
SUPERFAMILY = CATH_DOMAIN
SETUP["DOMAIN_PROSITE"] = DOMAIN_PROSITE
SETUP["PROSITE_DOMAIN"] = PROSITE_DOMAIN
SETUP["DOMAIN_CATH"] = DOMAIN_CATH
SETUP["CATH_DOMAIN"] = CATH_DOMAIN
SETUP["SUPERFAMILY"] = SUPERFAMILY

In [8]:
PROSITEFOLDER

'/home/user_stel/AISB/Project/databases/prosite/'

# Methods 

In [33]:
from time import sleep
from urllib.error import URLError


def selectUniquePerCluster(df, cathCluster, Uniref, withAlignment = True):
    """
    Return a datasert with only 1 data per choosed clusters.
    """
    
    if cathCluster not in ["S35","S60","S95","S100"]:
        raise ValueError('CathCluster given not in ["S35","S60","S95","S100"]')
    
    if Uniref not in ["uniref50","uniref90","uniref100"]:
        raise ValueError('CathCluster given not in ["uniref50","uniref90","uniref100"]')
    
    if withAlignment:
        df = df[~df.alignment_position.isnull()]
    
    cathdf = df.query("data_type == 'cathpdb'")
    seqdf = df.query("data_type == 'prosite'")
    
    def selectUniqueCath(group):
        uniqueNames = group.cathpdb.unique()
        select = uniqueNames[0]
        
        #return group.query("cathpdb == @select")
        return select
    
    def selectUniqueUniref(group,exclusion):
        uniqueNames = group.uniprot_acc.unique()
        select = uniqueNames[0]
        #return group.query("uniprot_acc == @select")
        if select not in exclusion:
            return select
        

    dfReprCathNames = cathdf.groupby(["domain",cathCluster]).apply(selectUniqueCath).to_numpy()
    
    excludeUniref = df.query("cathpdb in @dfReprCathNames").uniprot_acc.unique() #Structures are prior to sequences.
    dfReprUnirefNames = seqdf.groupby(["domain",Uniref]).apply(selectUniqueUniref, exclusion=excludeUniref).to_numpy()
    dfReprCath = cathdf.query("cathpdb in @dfReprCathNames")
    dfReprUniref = seqdf.query("uniprot_acc in @dfReprUnirefNames")
    
    return (pd.concat([dfReprCath,dfReprUniref]))

# download AlphaFold models 
def fetch_pdb_alfafold(uniprotids, domain):
    nomodels=[]
    withmodels=[]
    outfolder = f"{ALPHAFOLDFOLDER}/{domain}/raw"
    if not os.path.exists(outfolder):
        os.makedirs(outfolder)
        
    extractedfolder = f"{ALPHAFOLDFOLDER}/{domain}/extracted"
    if not os.path.exists(extractedfolder):
        os.makedirs(extractedfolder)
    else:
        if REBUILD == True: #delete extracted files
            files = glob.glob(f"{extractedfolder}/*.pdb")
            for f in files:
                os.remove(f)
    
    jsonfolder = f"{ALPHAFOLDFOLDER}/{domain}/json"
    if not os.path.exists(jsonfolder):
        os.makedirs(jsonfolder)

    for uniprot_id in tqdm(uniprotids, desc="Downloading "):
        url = f"https://alphafold.ebi.ac.uk/files/AF-{uniprot_id}-F1-model_v1.pdb"
        destination = f"{outfolder}/{uniprot_id}.pdb"
        if not os.path.isfile(destination): 
            try:
                urllib.request.urlretrieve(url, destination)
            except urllib.error.HTTPError as err:
                nomodels.append(uniprot_id)
                continue
        withmodels.append(uniprot_id)

    
    print(f"{len(nomodels)} out of {len(uniprotids)} without alfafold2 models ({len(nomodels)/len(uniprotids)*100:.2f}%)")
    return withmodels,nomodels

def get_prosite_boundaries_dict(domain):
    boundaries = {}
    prosite_ids = DOMAIN_PROSITE[domain]
    if type(prosite_ids) != type([]):
        prosite_ids = [prosite_ids]
    for msafile in prosite_ids:
        msafilepath = f"{PROSITEFOLDER}/msa/{msafile}.msa"
        msa = AlignIO.read(msafilepath,'fasta')
        for record in msa:
            seqid = record.id
            match = REGEX.match(seqid)
            if match:
                uniprot_id = match.group(2)
                start = match.group(3)
                end = match.group(4)
                boundaries[uniprot_id] = (int(start),int(end))
    return boundaries

def get_json(uniprot_acc, domain, source='ssf'):
    def request_URL(link, trial=1):
        try:
            response = requests.get(link).text
            return response
        except URLError as e:
            print(e, link)
            if trial >3 :
                print('3rd fail, skipping this one.')
                return None
            else:
                print(f"Trial {trial}, waiting 10s and trying again")
                sleep(10)
                return request_URL(link, trial=trial+1)
            
            
    jsonfolder = f"{ALPHAFOLDFOLDER}/{domain}/json"
    if not os.path.exists(jsonfolder):
        os.makedirs(jsonfolder)
        
    jsonfile = f"{jsonfolder}/{uniprot_acc}.json"
    if os.path.isfile(jsonfile):
        f = open(jsonfile)
        interpro = json.load(f)
    else:
        #make the query on ebi/interpro
        response = request_URL(f"https://www.ebi.ac.uk/interpro/api/entry/{source}/protein/reviewed/{uniprot_acc}/?page_size=200")
        if response == None:
            return None
        try:
            interpro = json.loads(response)
        except:
            print(f"no data for {uniprot_acc}.")
            return None
        with open(jsonfile,'w') as out:
            json.dump(interpro, out, indent=2)
            
    return(interpro)

def get_domain_fragment_query(uniprot_acc, domain, boundaries_prosite):
    start_PS,end_PS = boundaries_prosite[uniprot_acc]
    starts_ends = [boundaries_prosite[uniprot_acc]]

    if DOMAIN_INTERPRO_REFINE[domain] == True:
        if domain == "PLA":
            source = 'cathgene3d'
        else:
            source = 'ssf'
        interpro = get_json(uniprot_acc, domain, source)
        if interpro == None:
            return None
        QueryString = None
        
        for result in interpro["results"]:
            if result["metadata"]["accession"] == DOMAIN_INTERPRO[domain]:
                entry_protein_locations = result["proteins"][0]["entry_protein_locations"]
                for entry in entry_protein_locations: #Get the number of truncation in the domain.
                    nfrag = len(entry['fragments'])
                    
                    if domain == 'PLA': #Special case for PLA, we will ignore PROSITE annotation that are actually wrong.
                        frag = entry['fragments'][0] #Get first monomer only
                        s = entry['fragments'][0].get('start')
                        e = entry['fragments'][0].get('end')
                        starts_ends = [[s,e]]
                    else:
                        if nfrag >= 2 and ( entry['fragments'][0].get('start') - 50 <= start_PS <= entry['fragments'][0].get('start')+50) : #if truncated domain AND correspond to the prosite domain
                            print(f"splitting {domain}-{uniprot_acc}")
                            queries = []
                            starts_ends = []
                            for frag in entry['fragments']:
                                s=int(frag.get("start"))
                                e=int(frag.get("end"))
                                starts_ends.append([s,e])
                            if use_uniprot_boundaries == True:
                                starts_ends[0][0] = start_PS
                                starts_ends[-1][-1] = end_PS

                        else: #use prosite fragment
                            starts_ends = [[start_PS, end_PS]]
                    

                QueryString = " or ".join([f"({x} <= residue_number <= {y})" for x,y in starts_ends])
        
    else:
        QueryString = " or ".join([f"({x} <= residue_number <= {y})" for x,y in starts_ends])
    
    return QueryString


# Download CATH-domain-list

In [19]:
from pathlib import Path
import os, urllib.request
import pandas as pd

# ─── 1) Settings ───────────────────────────────────────────────────────────────
UPDATE     = False
CATHFOLDER = "/home/user_stel/AISB/Project/databases/cath/"
domfile    = "cath-domain-list-v4_2_0.txt"
url        = (
    "ftp://orengoftp.biochem.ucl.ac.uk/"
    "cath/releases/all-releases/v4_2_0/"
    "cath-classification-data/cath-domain-list-v4_2_0.txt"
)

# ─── 2) Ensure folder exists ───────────────────────────────────────────────────
os.makedirs(CATHFOLDER, exist_ok=True)

# ─── 3) Download (if needed) ──────────────────────────────────────────────────
destination = Path(CATHFOLDER) / domfile
if not destination.exists() or UPDATE:
    print(f"↓ Downloading CATH domain list to {destination}")
    urllib.request.urlretrieve(url, destination)

# ─── 4) Read into pandas ───────────────────────────────────────────────────────
column_names = [
    'Domain','Class','Architecture','Topology','Homologous',
    'S35','S60','S95','S100','S100Count','DomSize','Resolution'
]

df_cath = pd.read_csv(
    destination,
    sep=r'\s+',        # whitespace-delimited
    header=None,
    names=column_names,
    comment='#',       # skip lines beginning with “#”
    engine='python'
)

print(df_cath)


         Domain  Class  Architecture  Topology  Homologous  S35  S60  S95  \
0       1oaiA00      1            10         8          10    1    1    1   
1       1go5A00      1            10         8          10    1    1    1   
2       3frhA01      1            10         8          10    2    1    1   
3       3friA01      1            10         8          10    2    1    1   
4       3b89A01      1            10         8          10    2    1    1   
...         ...    ...           ...       ...         ...  ...  ...  ...   
434852  2kn1A00      4            10      1290          10    2    1    1   
434853  1vprA01      4            10      1300          10    1    1    1   
434854  1vprA02      4            10      1310          10    1    1    1   
434855  1jyoE00      4            10      1330          10    1    1    1   
434856  1jyoF00      4            10      1330          10    1    1    1   

        S100  S100Count  DomSize  Resolution  
0          1          1     

### Download Correspondance between Uniprot and PDB code

In [20]:
import os, requests
from time import sleep

url = "ftp://ftp.ebi.ac.uk/pub/databases/msd/sifts/flatfiles/csv/pdb_chain_uniprot.csv.gz"
destination = os.path.join(CATHFOLDER, "pdb_chain_uniprot.csv.gz")

def download_with_retries(url, dest, max_tries=3, chunk_size=1024*1024):
    for attempt in range(1, max_tries+1):
        try:
            with requests.get(url, stream=True, timeout=60) as r:
                r.raise_for_status()
                with open(dest, "wb") as f:
                    for chunk in r.iter_content(chunk_size=chunk_size):
                        if chunk:
                            f.write(chunk)
            return
        except Exception as e:
            print(f"↻ Attempt {attempt} failed: {e}")
            if attempt == max_tries:
                raise
            sleep(5)

# download only if missing or forced
if not os.path.exists(destination) or UPDATE:
    download_with_retries(url, destination)
    print("✅ Download complete:", destination)
else:
    print("✔️  Already downloaded:", destination)


✔️  Already downloaded: /home/user_stel/AISB/Project/databases/cath/pdb_chain_uniprot.csv.gz


In [21]:
import gzip, shutil

gz_path  = "/home/user_stel/AISB/Project/databases/uniprot/pdb_chain_uniprot.csv.gz"
csv_path = gz_path[:-3]

try:
    with gzip.open(gz_path, "rb") as f_in, open(csv_path, "wb") as f_out:
        shutil.copyfileobj(f_in, f_out)
except EOFError:
    print("⚠️ Warning: EOFError raised, file may be slightly truncated but proceeding anyway.")

print("✅ Decompressed to:", csv_path)



⚠️ Warning: EOFError raised, file may be slightly truncated but proceeding anyway.
✅ Decompressed to: /home/user_stel/AISB/Project/databases/uniprot/pdb_chain_uniprot.csv


# Download Prosite files 

In [22]:
import os, urllib.request, tarfile

PROSITE_URL      = "ftp://ftp.expasy.org/databases/prosite/prosite_alignments.tar.gz"
PROSITEFOLDER    = "/home/user_stel/AISB/Project/databases/prosite/"
archive_path     = os.path.join(PROSITEFOLDER, "prosite_alignments.tar.gz")
prosite_alignments       = os.path.join(PROSITEFOLDER, "msa")

# Only download & extract if the msa folder doesn’t already exist:
if not os.path.isdir(prosite_alignments):
    print(f"↓ Downloading PROSITE alignments to {archive_path}")
    urllib.request.urlretrieve(PROSITE_URL, archive_path)

    print("→ Extracting…")
    with tarfile.open(archive_path, "r:gz") as tf:
        tf.extractall(path=PROSITEFOLDER)

    # Rename the extracted folder to “msa”
    os.rename(
        os.path.join(PROSITEFOLDER, "prosite_alignments"),
        prosite_alignments
    )

    # Clean up
    os.remove(archive_path)
    print("✅ PROSITE data ready in", prosite_alignments)
else:
    print("✅ PROSITE data already present in", prosite_alignments)


✅ PROSITE data already present in /home/user_stel/AISB/Project/databases/prosite/msa


# Download CATH PDB files

In [23]:
domfile = os.path.join(CATHFOLDER, "cath-domain-list-v4_2_0.txt")
column_names = [
    'Domain','Class','Architecture','Topology','Homologous',
    'S35','S60','S95','S100','S100Count','DomSize','resolution',
]
df_domains = pd.read_csv(
    domfile,
    comment='#', sep=r"\s+", header=None, names=column_names, engine='python'
)
df_domains['Superfamily'] = df_domains.progress_apply(
    lambda x: f"{x.Class}.{x.Architecture}.{x.Topology}.{x.Homologous}",
    axis=1
)

# ─── 2) Group your domains by superfamily ────────────────────────────────────
domains_per_sf = (
    df_domains
      .groupby('Superfamily')['Domain']
      .apply(list)
      .to_dict()
)

# ─── 3) Download function ────────────────────────────────────────────────────
import os
import pandas as pd
from pathlib import Path
import urllib.request
from tqdm.auto import tqdm

# 1) Your mapping of superfamily name → CATH class.arch.topo.homologous
DOMAIN_CATH = {
    "PH":     "2.30.29.30",  # no
    "C2":     "2.60.40.150", # no
    "C1":     "3.30.60.20",  # no 
    "PX":     "3.30.1520.10",# no
    "FYVE":   "3.30.40.10",  # no
    "BAR":    "1.20.1270.60",# yes membrane-binding, signaling
    "ENTH":   "1.25.40.90",  # yes signaling 
    "SH2":    "3.30.505.10", # yes metabolism, growth
    "SEC14":  "3.40.525.10", # yes metabolism, transfering, energy balance 
    "START":  "3.30.530.20", # no
    "C2DIS":  "2.60.120.260",# no
    "GLA":    "2.40.20.10",  # yes signaling
    "PLD":    "3.20.20.190", # no
    "PLA":    "1.20.90.10",  # no
    "ANNEXIN":"1.10.220.10", # no
}

# 2) Read the domain list and compute a “Superfamily” column
domfile = "/home/user_stel/AISB/Project/databases/cath/cath-domain-list-v4_2_0.txt"
cols   = ['Domain','Class','Architecture','Topology','Homologous',
          'S35','S60','S95','S100','S100Count','DomSize','Resolution']
df_domains     = pd.read_csv(domfile, sep=r'\s+', header=None, names=cols, comment='#', engine='python')
df_domains['Superfamily'] = (
    df_domains.Class.astype(str) + '.' +
    df_domains.Architecture.astype(str) + '.' +
    df_domains.Topology.astype(str) + '.' +
    df_domains.Homologous.astype(str)
)

# 3) Keep only the rows whose Superfamily is in our DOMAIN_CATH values
wanted = set(DOMAIN_CATH.values())
df_domains = df_domains[df_domains.Superfamily.isin(wanted)]

# 4) Invert DOMAIN_CATH so we can look up the name from the string
inv = {v:k for k,v in DOMAIN_CATH.items()}

# 5) Build a dict superfamily → list of domain IDs
domains_per_sf = (
    df_domains.groupby('Superfamily')
      .Domain
      .apply(list)
      .rename(index=inv)   # turn the index from the string back to PH, C2, etc.
      .to_dict()
)

# 6) For each superfamily, download *only* those domains
CATHFOLDER = "/home/user_stel/AISB/Project/databases/cath/"
for sf, domlist in domains_per_sf.items():
    raw_dir   = Path(CATHFOLDER) / "domains" / sf / "raw"
    clean_dir = Path(CATHFOLDER) / "domains" / sf / "cleaned"
    raw_dir.mkdir(parents=True, exist_ok=True)
    clean_dir.mkdir(parents=True, exist_ok=True)

    for dom in tqdm(domlist, desc=sf):
        url  = f"http://www.cathdb.info/version/v4_2_0/api/rest/id/{dom}.pdb"
        dst  = raw_dir / f"{dom}.pdb"
        if not dst.exists():
            urllib.request.urlretrieve(url, dst.as_posix())


  0%|          | 0/434857 [00:00<?, ?it/s]

ANNEXIN:   0%|          | 0/405 [00:00<?, ?it/s]

BAR:   0%|          | 0/134 [00:00<?, ?it/s]

PLA:   0%|          | 0/505 [00:00<?, ?it/s]

ENTH:   0%|          | 0/131 [00:00<?, ?it/s]

...

FYVE:   0%|          | 0/605 [00:00<?, ?it/s]

SH2:   0%|          | 0/661 [00:00<?, ?it/s]

START:   0%|          | 0/445 [00:00<?, ?it/s]

C1:   0%|          | 0/128 [00:00<?, ?it/s]

SEC14:   0%|          | 0/53 [00:00<?, ?it/s]

In [24]:
# Reading Cath domain list
df_domains = pd.read_csv(domfile,comment='#', sep=r"\s+", header=None)
df_domains.columns = column_names
if PARRALEL:
    df_domains['Superfamily'] = df_domains.parallel_apply(lambda x: f"{x.Class}.{x.Architecture}.{x.Topology}.{x.Homologous}", axis=1)
else:
    df_domains['Superfamily'] = df_domains.progress_apply(lambda x: f"{x.Class}.{x.Architecture}.{x.Topology}.{x.Homologous}", axis=1)

In [25]:
# Creating the superfamily 
cathSuperFamily = pd.DataFrame()
cathSuperFamily['Superfamily'] = df_domains.Superfamily
cathSuperFamily['Domain'] = df_domains.Domain
print(df_domains)

         Domain  Class  Architecture  Topology  Homologous  S35  S60  S95  \
0       1oaiA00      1            10         8          10    1    1    1   
1       1go5A00      1            10         8          10    1    1    1   
2       3frhA01      1            10         8          10    2    1    1   
3       3friA01      1            10         8          10    2    1    1   
4       3b89A01      1            10         8          10    2    1    1   
...         ...    ...           ...       ...         ...  ...  ...  ...   
434852  2kn1A00      4            10      1290          10    2    1    1   
434853  1vprA01      4            10      1300          10    1    1    1   
434854  1vprA02      4            10      1310          10    1    1    1   
434855  1jyoE00      4            10      1330          10    1    1    1   
434856  1jyoF00      4            10      1330          10    1    1    1   

        S100  S100Count  DomSize  resolution   Superfamily  
0          1  

In [26]:
CATHVERSION = 'v4_2_0'

# Download AlphaFold Models

In [27]:
### Parse your PROSITE MSAs for UniProt IDs 
import re, glob
from Bio import SeqIO

PROSITE_MSA_DIR = "/home/user_stel/AISB/Project/databases/prosite/msa"
PROSITE_HEADER_RX = re.compile(r"^>[^|]+\|([^/]+)/")

def get_uniprot_from_msa(msa_path):
    # grab just the first header line
    with open(msa_path) as fh:
        for line in fh:
            if line.startswith(">"):
                m = PROSITE_HEADER_RX.match(line)
                return m.group(1)
    return None

prosite_ids = []
for msa in glob.glob(f"{PROSITE_MSA_DIR}/*.msa"):
    uid = get_uniprot_from_msa(msa)
    if uid:
        prosite_ids.append(uid)

prosite_ids = list(set(prosite_ids))
print(f"Found {len(prosite_ids)} UniProt IDs in PROSITE MSAs")


Found 2353 UniProt IDs in PROSITE MSAs


In [28]:
# Read your CATH→UniProt mapping
import pandas as pd

mapping_csv = "/home/user_stel/AISB/Project/databases/cath/pdb_chain_uniprot.csv"
m = pd.read_csv(mapping_csv, comment="#")
cath_ids = m["SP_PRIMARY"].dropna().unique().tolist()
print(f"Found {len(cath_ids)} UniProt IDs in CATH mapping")

Found 46780 UniProt IDs in CATH mapping


In [34]:
# combine both
uniprot_ids = list(set(prosite_ids) | set(cath_ids))
print(f"Total unique UniProt IDs to fetch from AlphaFold: {len(uniprot_ids)}")

Total unique UniProt IDs to fetch from AlphaFold: 48770


In [ ]:
for domain in SUPERFAMILY.keys():
    # 1) Download full-length AF models for this domain
    withmodels, nomodels = fetch_pdb_alfafold(uniprot_ids, domain=domain)
    print(f"[{domain}] fetched {len(withmodels)} / {len(withmodels)+len(nomodels)}")

    # 2) Build the slice query for this domain
    boundaries = get_prosite_boundaries_dict(domain)
    query = get_domain_fragment_query(domain, boundaries)

    # 3) Trim & save each model
    raw_dir       = f"{ALPHAFOLDFOLDER}/{domain}/raw"
    extracted_dir = f"{ALPHAFOLDFOLDER}/{domain}/extracted"
    os.makedirs(extracted_dir, exist_ok=True)

    for uid in withmodels:
        src = f"{raw_dir}/{uid}.pdb"
        dst = f"{extracted_dir}/{uid}.pdb"
        pp = PandasPdb().read_pdb(src)
        pp.df["ATOM"] = pp.df["ATOM"].query(query)
        pp.to_pdb(dst)


In [37]:
REBUILD = True
use_uniprot_boundaries = True
use_all_AFmodels = True

# 2️⃣ Download full-length AF PDBs:
withmodels, nomodels = fetch_pdb_alfafold(uniprot_ids, domain="PH")
print(f"✅ {len(withmodels)} models fetched, {len(nomodels)} missing")

# 3️⃣ Compute “in-domain” slice query:
boundaries_prosite = get_prosite_boundaries_dict("PH")
query = get_domain_fragment_query("PH", boundaries_prosite)
#    e.g. "(45 <= residue_number <= 130) or (200 <= residue_number <= 270)"

raw_dir      = f"{ALPHAFOLDFOLDER}/PH/raw"
extracted_dir= f"{ALPHAFOLDFOLDER}/PH/extracted"

os.makedirs(extracted_dir, exist_ok=True)

for uid in withmodels:
    src = f"{raw_dir}/{uid}.pdb"
    dst = f"{extracted_dir}/{uid}.pdb"
    pp = PandasPdb().read_pdb(src)
    pp.df["ATOM"] = pp.df["ATOM"].query(query)
    pp.to_pdb(dst)

IncompleteRead: IncompleteRead(0 bytes read)

# Generation Phase

In [73]:
RECALCULATION = True 

In [74]:
recalculation_widget = widgets.ToggleButton(
    value=RECALCULATION,
    description='Recalculation ?',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click for recalculation',
    icon='cogs' # (FontAwesome names without the `fa-` prefix)
)
display(recalculation_widget)

ToggleButton(value=True, description='Recalculation ?', icon='cogs', tooltip='Click for recalculation')

### Instanciate the builder object

In [75]:
import pepr2ds.builder.Builder as builderEngine
importlib.reload(builderEngine)
builder = builderEngine.Builder(SETUP, recalculate = recalculation_widget.value, update=False, notebook = True, core=1)

<module 'pepr2ds.builder.Builder' from '/home/user_stel/AISB/Project/pepr2ds/builder/Builder.py'>

### Cleaning -> Populate the cleaned directories 

In [76]:
builder.structure.clean_all_pdbs()

cleaning:   0%|          | 0/5916 [00:00<?, ?it/s]

In [ ]:
# Verify that the **builder.structure.clean_all_pdbs()** worked 

raw = glob.glob(os.path.join(SETUP["CATHFOLDER"], "domains", "*", "raw", "*.pdb"))
cleaned = glob.glob(os.path.join(SETUP["CATHFOLDER"], "domains", "*", "cleaned", "*.pdb"))

print(f"Found {len(raw)} raw PDBs and {len(cleaned)} cleaned PDBs.")

# it should display same number or raw PDBs and cleaned ones 


Found 5916 raw PDBs and 5916 cleaned PDBs.


### Generate Dataset

In [ ]:
df_struc = builder.structure.build_structural_dataset()